Import primary data-structure OR primary plus additional parameters calculated during preprocessing

In [1]:
# As this is faster than loading the full data-structure, the seperation saves time on partial analysis and debugging
# full data-structure, initial parameter and covariate sets
%run fMRI_prestats_data-struct.ipynb

subject(s) missing!
First line of list-alignment:   FED006 FED006 FED007
Control relevant files!


 The following subjects where excluded from further analysis due to false file numbers or missing data:  
 EPI:  ['FED006'] T1:  ['FED006']
The new subject list after initial data and image file checkup:

['FED007', 'FED008', 'FED009', 'FED010', 'FED011', 'FED012', 'FED013', 'FED014', 'FED015', 'FED016', 'FED017', 'FED018', 'FED019', 'FED020', 'FED021', 'FED022', 'FED023', 'FED024', 'FED025', 'FED026', 'FED027', 'FED028', 'FED029', 'FED030', 'FED031', 'FED032', 'FED033', 'FED034', 'FED035', 'FED036', 'FED037', 'FED038', 'FED039', 'FED040', 'FED041', 'FED042', 'FED043', 'FED044', 'FED045', 'FED046', 'FED047', 'FED048', 'FED049', 'FED050', 'FED051', 'FED052', 'FED053', 'FED054', 'FED055', 'FED056', 'FED057', 'FED058', 'FED059', 'FED060', 'FED061', 'FED062', 'FED063', 'FED064', 'FED065', 'FED066', 'FED067', 'FED068']


That leaves a total of  62 remaining subjects.


The following parameters

In [2]:
# full data-structure, parameter and covariate sets including parameters extracted during preprocessing
%run fMRI_prestats_data-struct2-prepadds.ipynb

subject(s) missing!
First line of list-alignment:   FED006 FED006 FED007
Control relevant files!


 The following subjects where excluded from further analysis due to false file numbers or missing data:  
 EPI:  ['FED006'] T1:  ['FED006']
The new subject list after initial data and image file checkup:

['FED007', 'FED008', 'FED009', 'FED010', 'FED011', 'FED012', 'FED013', 'FED014', 'FED015', 'FED016', 'FED017', 'FED018', 'FED019', 'FED020', 'FED021', 'FED022', 'FED023', 'FED024', 'FED025', 'FED026', 'FED027', 'FED028', 'FED029', 'FED030', 'FED031', 'FED032', 'FED033', 'FED034', 'FED035', 'FED036', 'FED037', 'FED038', 'FED039', 'FED040', 'FED041', 'FED042', 'FED043', 'FED044', 'FED045', 'FED046', 'FED047', 'FED048', 'FED049', 'FED050', 'FED051', 'FED052', 'FED053', 'FED054', 'FED055', 'FED056', 'FED057', 'FED058', 'FED059', 'FED060', 'FED061', 'FED062', 'FED063', 'FED064', 'FED065', 'FED066', 'FED067', 'FED068']


That leaves a total of  62 remaining subjects.


The following parameters

KeyboardInterrupt: 

KeyboardInterrupt: 

In [3]:
# control results
key = random.sample(data.keys(), 1)
print(data[key[0]])
print(data["FED015"])

{'files': ['/fMRI/FED026/FMRI_0010_20150211171144.json', '/fMRI/FED026/FMRI_0010_20150211171144.nii', '/fMRI/FED026/FMRI_0010_20150211171144_distcor.nii', '/fMRI/FED026/FMRI_distcor_effect.nii', '/fMRI/FED026/FMRI_mc-ri-s5-cr-m-in.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5-cr-m-in2x-hp.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5-cr-m-in2x.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5-cr-m.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5-cr.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5-cr_mean.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s5.nii.gz_cr-matrix', '/fMRI/FED026/FMRI_mc-ri-s5.nii.gz_usan_size.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s8.nii.gz', '/fMRI/FED026/FMRI_mc-ri-s8.nii.gz_usan_size.nii.gz', '/fMRI/FED026/FMRI_mc-ri.nii.gz', '/fMRI/FED026/FMRI_mc-ri_mean.nii.gz', '/fMRI/FED026/FMRI_mc.nii.gz', '/fMRI/FED026/FMRI_mc.nii.gz.par', '/fMRI/FED026/FMRI_mc.nii.gz_disp.png', '/fMRI/FED026/FMRI_mc_mask.nii.gz', '/fMRI/FED026/FMRI_mc_masked.nii.gz', '/fMRI/FED026/FMRI_mc_mean.nii.gz', '/fMRI/FED026/FMRI

 Pre-configurations

In [ ]:
# Re-activate nipype's INFO output for interfaces and workflows , as we want to see what's going on
logging.getLogger('nipype.interface').setLevel("INFO")
logging.getLogger('nipype.workflow').setLevel("INFO")

In [134]:
# set default data ouput for all FSL operations
fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# 1st-level modelling

# Post-Stats

Corregistration to MNI space ("spatial normalisation"): FSL FEAT - 2nd step (post-stat)

# QC